# Example of using LLM-based information retrieval for recommendation task

Run in google colaboratory

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
import os

repo_path = '/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec'
sys.path.append(repo_path)

## Install requirements

In [3]:
!pip install -q -r '{repo_path}/requirements/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.6/274.6 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

## Add config

In [21]:
import os

config_dict = {
    "csv_args": {"delimiter": "\t"},
    "source_column": "item_id:token",
    "search_kwargs": {"k": 200},
    "data_path": os.path.join(repo_path, "datasets"),
    "load_col": {
        "inter": ["user_id", "item_id", "rating", "timestamp"],
        "item": ["item_id", "movie_title"],
    },
    "text_col": ["movie_title", "release_year", "class"],
    "MAX_ITEM_LIST_LENGTH": 10,
    "eval_args": {"split": {"LS": "valid_and_test"}, "order": "TO", "mode": "full"},
    "repeatable": True,
    "loss_type": "CE",
    "train_batch_size": 100,
    "eval_batch_size": 8,
    "valid_metric": "NDCG@10",
    "metrics": ["Recall", "NDCG"],
    "topk": [1, 20, 50, 100, 200],
    "train_neg_sample_args": None,
}

In [27]:
assert config_dict['search_kwargs']['k'] >= max(config_dict['topk'])

## Get dataset and config

In [28]:
from llm4rec.tasks import RecBoleRetrievalRecommender
from llm4rec.dataset import RecboleSeqDataset
from llm4rec.trainer import PipelineTrainer
from recbole.data.utils import data_preparation
from recbole.config import Config
from recbole.model.abstract_recommender import AbstractRecommender
import os
import torch

model_cls = RecBoleRetrievalRecommender
dataset_name = 'ml-100k'

config = Config(model=model_cls, dataset=dataset_name,
            config_dict=config_dict)

dataset = RecboleSeqDataset(config)
_, _, test_data = data_preparation(config, dataset)

## Case 1: open source information retrieval using HuggingFace Sentence Transformers model

In [29]:
model = model_cls(config=config,
                embeddings=None,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                source_column=config['source_column'],
                csv_args=config['csv_args'],
                text_splitter_args=dict(chunk_size=1000, chunk_overlap=0),
                search_type="similarity",
                search_kwargs=config['search_kwargs'],
                emb_model_name="all-MiniLM-L6-v2",
                emb_model_kwargs={"device":"cuda:0" if torch.cuda.is_available() else "cpu"},
                dataset=dataset)

trainer = PipelineTrainer(config, model)
test_result = trainer.evaluate(test_data, show_progress=config['show_progress'])

Evaluate   : 100%|████████████████████████████████████████████████| 118/118 [00:15<00:00,  7.45it/s]


In [30]:
test_result

OrderedDict([('recall@1', 0.0021),
             ('recall@20', 0.0583),
             ('recall@50', 0.1092),
             ('recall@100', 0.1771),
             ('recall@200', 0.2545),
             ('ndcg@1', 0.0021),
             ('ndcg@20', 0.0211),
             ('ndcg@50', 0.0311),
             ('ndcg@100', 0.0421),
             ('ndcg@200', 0.0528)])

## Case 2: OpenAI information retrieval using OpenAIEmbeddings

In [ ]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

path_to_openai_env = os.path.join(repo_path, "openai.env")
load_dotenv(path_to_openai_env)

openai_api_key = os.environ.get("API_KEY")
embeddings_model = OpenAIEmbeddings(
            openai_api_key=openai_api_key, model="text-embedding-ada-002"
        )
model = model_cls(config=config,
                embeddings=embeddings_model,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                source_column=config['source_column'],
                csv_args=config['csv_args'],
                text_splitter_args=dict(chunk_size=1000, chunk_overlap=0),
                search_type="similarity",
                search_kwargs=config['search_kwargs'],
                emb_model_name="",
                emb_model_kwargs=None,
                dataset=dataset)

trainer = PipelineTrainer(config, model)
test_result = trainer.evaluate(test_data, show_progress=config['show_progress'])

Evaluate   : 100%|██████████████████████████████████████████████████| 25/25 [01:30<00:00,  3.63s/it]


In [ ]:
test_result

OrderedDict([('recall@1', 0.035),
             ('recall@5', 0.07),
             ('recall@10', 0.085),
             ('recall@20', 0.125),
             ('ndcg@1', 0.035),
             ('ndcg@5', 0.0529),
             ('ndcg@10', 0.0578),
             ('ndcg@20', 0.0683)])